
# Validate BlockModel

This example demonstrates how to validate a pandas block model with a pandera schema prior to writing it to an OMF file.


In [ ]:
import shutil
import tempfile
from pathlib import Path

import pandas as pd

from omfpandas import OMFPandasReader, OMFPandasWriter

## Instantiate
Create the object OMFPandas with the path to the OMF file.



In [ ]:
test_omf_path: Path = Path('../assets/test_file.omf')

# create a temporary copy to preserve the original file
temp_omf_path: Path = Path(tempfile.gettempdir()) / 'test_file_copy.omf'
shutil.copy(test_omf_path, temp_omf_path)

# Display the head of the original block model
blocks: pd.DataFrame = OMFPandasReader(filepath=temp_omf_path).read_blockmodel(blockmodel_name='regular')
blocks.head()

## Write with Schema
The schema used to validate the dataframe is a Pandera schema.  The schema can be a YAML file (or Pandera compatible
dictionary) that is loaded by the Pandera library.  The schema is used to validate the dataframe before writing it
to the OMF file.



In [ ]:
omfpw: OMFPandasWriter = OMFPandasWriter(filepath=temp_omf_path)

In [ ]:
omfpw.create_blockmodel(blocks=blocks, blockmodel_name='regular',
                        pd_schema=test_omf_path.with_suffix('.schema.yaml'),
                        allow_overwrite=True)

The schema is persisted inside the omf file.  This enables the schema to be used to validate modifications.



In [ ]:
bm = omfpw.get_element_by_name('regular')
print(bm.metadata.get('pd_schema'))

## Demonstrate Failure
We'll demonstrate a failure by modifying the block model in a way that violates the schema.



In [ ]:
blocks['random attr'] = blocks['random attr'] * 2

try:
    omfpw.create_blockmodel(blocks=blocks, blockmodel_name='regular',
                            pd_schema=test_omf_path.with_suffix('.schema.yaml'),
                            allow_overwrite=True)
except Exception as e:
    print(e)

Update with valid data



In [ ]:
omfpw.write_blockmodel_attribute(blockmodel_name='regular', series=blocks['random attr'] / 4, allow_overwrite=True)

blocks: pd.DataFrame = OMFPandasReader(filepath=temp_omf_path).read_blockmodel(blockmodel_name='regular')
blocks.head()

## View the changelog



In [ ]:
omfpw.changelog

Clean up



In [ ]:
temp_omf_path.unlink()